In [238]:
import pandas as pd
import numpy as np
import re
from random import random
import emoji
from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import torch
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from nltk.tokenize import TweetTokenizer
from collections import Counter
import time

In [239]:
from simpletransformers.classification import MultiLabelClassificationModel, ClassificationModel
from sklearn.preprocessing import MultiLabelBinarizer

In [240]:
from simpletransformers.language_modeling import LanguageModelingModel

In [241]:
tknzr = TweetTokenizer()

## Read input data

In [242]:
df = pd.read_json('/home/ino/Projects/Yao_tmp/Intro-PR/NLP/NLP-2020-EmotionGIF/source/train_gold.json', lines=True)
df_new = df.copy()
df_new.head()

,idx,text,reply,categories,mp4
0,0,we can all agree that any song by Niall Horan.,oui oui,[yes],6dc39e96b11275f064fdaed88273b45e.mp4
1,1,Will you be installing #ScottyFromMarketing's ...,,[no],cfff051f05d8d3b7136c7d58ea6ad55f.mp4
2,2,Growing up my mum would call me a Nigga despit...,And he joins in??? Pour some hot grits on em,[smh],bf39e7bd9ad24354ce3ba6822b0104af.mp4
3,3,Rest your head on my chest when the world feel...,😂😂😂😂😂,[wink],173a707a04c277354a2f23cf01d6151e.mp4
4,4,Imagine Will Hernandez and Wills both doing a ...,,[yes],aab6d6bfb0c1382269ddba9b71cc8b7a.mp4


In [243]:
# # Try shuffle data
# df_new = df.sample(frac=1).reset_index(drop=True)
# df_new.head()

In [244]:
categories_type = pd.read_json('/home/ino/Projects/Yao_tmp/Intro-PR/NLP/NLP-2020-EmotionGIF/source/categories.json', lines=True)
categories_mapping = {v[0]: k for k, v in categories_type.to_dict('list').items()}

In [245]:
df_dev = pd.read_json('/home/ino/Projects/Yao_tmp/Intro-PR/NLP/NLP-2020-EmotionGIF/source/dev_unlabeled.json', lines=True)
df_dev_result = df_dev.copy()[['text', 'reply']]
df_dev_result.head()

,text,reply
0,"Drop your cash app, use hashtag #BailoutHumansNow",$tyratomaro #BailoutHumans
1,After interviewing with a few incredible peopl...,CONGRATS!!!!!
2,I know GTC festival not happening next month b...,
3,"Lordy, my daughter just said, “I wonder how th...",
4,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...


In [246]:
df_test = pd.read_json('/home/ino/Projects/Yao_tmp/Intro-PR/NLP/NLP-2020-EmotionGIF/source/test_unlabeled.json', lines=True)
df_test_result = df_test.copy()[['text', 'reply']]
df_test_result.head()

,text,reply
0,@Youngdeji_ I think if uzi and carti dropping ...,
1,For the third year in a row we’re discussing t...,
2,dababy album sounds like it was made for nigga...,That's why you bought it.
3,Majority of Indians don’t watch any sport othe...,@ZairaWasimmm got a great story because of the...
4,everybody is just now listening to @madisonbee...,


## Data cleaning and preprocessing

In [247]:
def tokenize(text):
    tokens = []
    for each_text in text:
        tokens.append(tknzr.tokenize(each_text.lower()))
    return tokens

In [248]:
def get_terms(token_list):
    vocabulary = Counter()
    for tokens in token_list:
        vocabulary.update(tokens)    
    return vocabulary

In [249]:
def get_idf(token_list):
    idf = []
    for token, freq in features_terms:
        count = 0
        for words in token_list:
            if token in words:
                count += 1
        idf.append(count)
    for i in range(len(idf)):
        idf[i] = math.log(len(token_list)/idf[i])
    return idf

In [250]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    for word, count in tf_high:
        tweet = tweet.replace(' ' + word + ' ', ' ')
    for word in tf_1:
        tweet = tweet.replace(' ' + word + ' ', ' ')
    return tweet

In [251]:
before = time.time()
text_list = df_new['text'].to_list()
tokens = tokenize(text_list)
features_terms = get_terms(tokens)
tf_high = features_terms.most_common(25)
tf_1 = [word for word, count in features_terms.most_common() if count==1]

df_new['text'] = df_new.text.apply(preprocess_tweet)
# # df_new['reply'] = df_new.reply.apply(preprocess_tweet)
df_dev_result['text'] = df_dev_result.text.apply(preprocess_tweet)
# df_dev_result['reply'] = df_dev_result.reply.apply(preprocess_tweet)
# print(df_new['text'][1])
# print(df_new['reply'][3])
after = time.time()
after - before

164.76767301559448

In [ ]:
df_new['text'][1]

### Use multi-hot encoding and change column name

In [252]:
mlb = MultiLabelBinarizer()
multi_hot = mlb.fit_transform(df_new['categories'].values)
multi_hot_list = [list(_) for _ in multi_hot]

In [253]:
df_new['labels'] = multi_hot_list
df_new = df_new[['text', 'reply', 'labels']]
df_new.columns = ['text_a', 'text_b', 'labels']
df_dev_result.columns = ['text_a', 'text_b']
df_test_result.columns = ['text_a', 'text_b']

## Training language model first, can imporve accuracy!

In [254]:
# Output data to train_file for training LM
df_forLM = df_new.copy()
# df_forLM['text'] = ['<s> '] + df_new['text_a'] + [' </s></s> '] + df_new['text_b'] + [' <s>']
df_forLM['text'] = ['[CLS] '] + df_new['text_a'] + [' [SEP] '] + df_new['text_b'] + [' [SEP]']
# df_forLM_dev = df_dev_result.copy()
# df_forLM_dev['text'] = ['<s> '] + df_dev_result['text_a'] + [' </s></s> '] + df_dev_result['text_b'] + [' <s>']
# df_forLM_test = df_test_result.copy()
# df_forLM_test['text'] = ['<s> '] + df_test_result['text_a'] + [' [SEP] '] + df_test_result['text_b']
# pd.concat([df_forLM['text'], df_forLM_dev['text'], df_forLM_test['text']])[1]

In [255]:
df_forLM['text'].to_csv(r'LM/training.txt', header=None, index=None, sep=' ')
# pd.concat([df_forLM['text'], df_forLM_dev['text']]).to_csv(r'LM/training_dev.txt', header=None, index=None, sep=' ')
# pd.concat([df_forLM['text'], df_forLM_dev['text'], df_forLM_test['text']]).to_csv(r'LM/training_dev_test.txt', header=None, index=None, sep=' ')

In [256]:
train_lm_args = {
    "output_dir": "LM/outputs_roberta_uncased_LM_training_pair/",
    "cache_dir": "cache/",
    "best_model_dir": "LM/outputs/best_model/",

    "fp16": False,
    "fp16_opt_level": "O1",
    "max_seq_length": 114,
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "gradient_accumulation_steps": 1,
    "num_train_epochs": 4,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "do_lower_case": False,
}
model = LanguageModelingModel("roberta", "roberta-base", args=train_lm_args)

I0602 18:45:11.784299 139786362763072 filelock.py:274] Lock 139779598689672 acquired on cache/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock
I0602 18:45:11.785942 139786362763072 file_utils.py:436] https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json not found in cache or force_download set to True, downloading to /home/ino/Projects/Kai_tmp/cache/tmp82kwn72b


I0602 18:45:13.866592 139786362763072 file_utils.py:440] storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json in cache at cache/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
I0602 18:45:13.866979 139786362763072 file_utils.py:443] creating metadata file for cache/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
I0602 18:45:13.867591 139786362763072 filelock.py:318] Lock 139779598689672 released on cache/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock


I0602 18:45:14.817342 139786362763072 filelock.py:274] Lock 139782237559328 acquired on cache/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
I0602 18:45:14.819007 139786362763072 file_utils.py:436] https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt not found in cache or force_download set to True, downloading to /home/ino/Projects/Kai_tmp/cache/tmp08_e1g1a


I0602 18:45:16.656232 139786362763072 file_utils.py:440] storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt in cache at cache/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
I0602 18:45:16.656765 139786362763072 file_utils.py:443] creating metadata file for cache/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
I0602 18:45:16.657276 139786362763072 filelock.py:318] Lock 139782237559328 released on cache/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
I0602 18:45:16.657529 139786362763072 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at cache/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41c

I0602 18:45:17.615552 139786362763072 filelock.py:274] Lock 139779598680584 acquired on cache/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
I0602 18:45:17.617280 139786362763072 file_utils.py:436] https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json not found in cache or force_download set to True, downloading to /home/ino/Projects/Kai_tmp/cache/tmp0z1wvtz9


I0602 18:45:18.547198 139786362763072 file_utils.py:440] storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json in cache at cache/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
I0602 18:45:18.547805 139786362763072 file_utils.py:443] creating metadata file for cache/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
I0602 18:45:18.548400 139786362763072 filelock.py:318] Lock 139779598680584 released on cache/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
I0602 18:45:18.548912 139786362763072 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at cache/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81

I0602 18:45:18.857625 139786362763072 filelock.py:274] Lock 139779591031384 acquired on cache/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock
I0602 18:45:18.859058 139786362763072 file_utils.py:436] https://cdn.huggingface.co/roberta-base-pytorch_model.bin not found in cache or force_download set to True, downloading to /home/ino/Projects/Kai_tmp/cache/tmpm7yvlkwv


I0602 18:45:24.019864 139786362763072 file_utils.py:440] storing https://cdn.huggingface.co/roberta-base-pytorch_model.bin in cache at cache/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
I0602 18:45:24.020266 139786362763072 file_utils.py:443] creating metadata file for cache/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
I0602 18:45:24.020787 139786362763072 filelock.py:318] Lock 139779591031384 released on cache/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock
I0602 18:45:24.021033 139786362763072 modeling_utils.py:650] loading weights file https://cdn.huggingface.co/roberta-base-pytorch_model.bin from cache at cache/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3f

I0602 18:45:27.818131 139786362763072 modeling_utils.py:741] Weights of RobertaForMaskedLM not initialized from pretrained model: ['lm_head.decoder.bias']


In [257]:
before = time.time()
model.train_model('LM/training.txt')
after = time.time()
after - before

I0602 18:45:27.930772 139786362763072 language_modeling_utils.py:181]  Creating features from dataset file at cache/


I0602 18:45:32.725187 139786362763072 language_modeling_utils.py:229]  Saving features into cached file cache/roberta_cached_lm_111_training.txt
I0602 18:45:32.757880 139786362763072 language_modeling_model.py:473]  Training started


Running loss: 2.680486

I0602 18:48:26.711689 139786362763072 configuration_utils.py:144] Configuration saved in LM/outputs_roberta_uncased_LM_training_pair/checkpoint-752-epoch-1/config.json


I0602 18:48:26.977880 139786362763072 modeling_utils.py:483] Model weights saved in LM/outputs_roberta_uncased_LM_training_pair/checkpoint-752-epoch-1/pytorch_model.bin


...

Running loss: 1.568282

I0602 18:57:14.161850 139786362763072 configuration_utils.py:144] Configuration saved in LM/outputs_roberta_uncased_LM_training_pair/checkpoint-3008-epoch-4/config.json


I0602 18:57:14.408723 139786362763072 modeling_utils.py:483] Model weights saved in LM/outputs_roberta_uncased_LM_training_pair/checkpoint-3008-epoch-4/pytorch_model.bin
I0602 18:57:15.160012 139786362763072 configuration_utils.py:144] Configuration saved in LM/outputs_roberta_uncased_LM_training_pair/config.json


I0602 18:57:15.409447 139786362763072 modeling_utils.py:483] Model weights saved in LM/outputs_roberta_uncased_LM_training_pair/pytorch_model.bin
I0602 18:57:15.459530 139786362763072 language_modeling_model.py:402]  Training of roberta model complete. Saved to LM/outputs_roberta_uncased_LM_training_pair/.


707.6288330554962

## Training model

In [258]:
train_args = {
    "output_dir": "model_results_gpu/outputs_roberta_training/",
    "cache_dir": "cache/",
    "best_model_dir": "model_results_gpu/outputs/best_model/",

    "fp16": False,
    "fp16_opt_level": "O1",
    "max_seq_length": 114,
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "gradient_accumulation_steps": 1,
    "num_train_epochs": 4,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "do_lower_case": False,

    "logging_steps": 50,
    "evaluate_during_training": False,
    "evaluate_during_training_steps": 2000,
    "evaluate_during_training_verbose": False,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": False,
    "save_steps": 2000,
    "no_cache": True,
    "save_model_every_epoch": True,
    "tensorboard_dir": None,

    "overwrite_output_dir": False,
    "reprocess_input_data": True,

    "n_gpu": 1,
    "silent": False,
    "use_multiprocessing": False,

    "wandb_project": None,
    "wandb_kwargs": {},

    "use_early_stopping": True,
    "early_stopping_patience": 3,
    "early_stopping_delta": 0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,

    "manual_seed": None,
    "encoding": None,
    "config": {}
}

In [259]:
# model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=43, args=train_args)
# Read from pretrained language model
model = MultiLabelClassificationModel('roberta', 'LM/outputs_roberta_uncased_LM_training_pair', num_labels=43, args=train_args)

I0602 19:00:08.192695 139786362763072 configuration_utils.py:283] loading configuration file LM/outputs_roberta_uncased_LM_training_pair/config.json
I0602 19:00:08.193446 139786362763072 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
  

In [260]:
before = time.time()
model.train_model(df_new)
after = time.time()
after - before

I0602 19:00:12.768683 139786362763072 classification_model.py:801]  Converting to features started. Cache is not used.


I0602 19:00:18.147119 139786362763072 classification_model.py:367]    Starting fine-tuning.


Running loss: 0.126159

I0602 19:05:51.608978 139786362763072 configuration_utils.py:144] Configuration saved in model_results_gpu/outputs_roberta_training/checkpoint-2000/config.json
I0602 19:05:51.848461 139786362763072 modeling_utils.py:483] Model weights saved in model_results_gpu/outputs_roberta_training/checkpoint-2000/pytorch_model.bin
I0602 19:05:52.375119 139786362763072 configuration_utils.py:144] Configuration saved in model_results_gpu/outputs_roberta_training/checkpoint-2000-epoch-1/config.json


I0602 19:05:52.617387 139786362763072 modeling_utils.py:483] Model weights saved in model_results_gpu/outputs_roberta_training/checkpoint-2000-epoch-1/pytorch_model.bin


...

Running loss: 0.113842

I0602 19:22:39.984638 139786362763072 configuration_utils.py:144] Configuration saved in model_results_gpu/outputs_roberta_training/checkpoint-8000/config.json
I0602 19:22:40.226206 139786362763072 modeling_utils.py:483] Model weights saved in model_results_gpu/outputs_roberta_training/checkpoint-8000/pytorch_model.bin
I0602 19:22:40.756170 139786362763072 configuration_utils.py:144] Configuration saved in model_results_gpu/outputs_roberta_training/checkpoint-8000-epoch-4/config.json


I0602 19:22:40.997750 139786362763072 modeling_utils.py:483] Model weights saved in model_results_gpu/outputs_roberta_training/checkpoint-8000-epoch-4/pytorch_model.bin
I0602 19:22:41.719073 139786362763072 configuration_utils.py:144] Configuration saved in model_results_gpu/outputs_roberta_training/config.json


I0602 19:22:41.961348 139786362763072 modeling_utils.py:483] Model weights saved in model_results_gpu/outputs_roberta_training/pytorch_model.bin
I0602 19:22:42.006997 139786362763072 classification_model.py:279]  Training of roberta model complete. Saved to model_results_gpu/outputs_roberta_training/.


1349.372787475586

### Load model (unused can comment)

In [261]:
# model = MultiLabelClassificationModel('roberta', 'model_results_gpu/outputs_roberta_#64/checkpoint-6000-epoch-3')

### Evaluating model for LRAP and mAP

In [262]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(df_new)
print(result)


{'LRAP': 0.5364130718472097, 'eval_loss': 0.09930125410854816}


In [263]:
avg_map = 0
total_predict = []

for eid, each_outputs in enumerate(model_outputs):
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(model_outputs[eid])), key=lambda k: model_outputs[eid][k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            if key in df['categories'][eid]:
                correct += 1
            predict_class.append(key)
    avg_map += (correct / len(df['categories'][eid]))
    total_predict.append(predict_class)

avg_map /= len(df['categories'])
print("avg map = {}".format(avg_map))

avg map = 0.6800333333333327


## Predict results [pair sentance](https://simpletransformers.ai/docs/classification-data-formats/)

In [264]:
# text + [SEP] + reply
to_predict = []
for i in range(len(df_dev_result)):
    # for roberta format
    text_reply = '<s> ' + df_dev_result['text_a'][i] + ' </s></s> ' + df_dev_result['text_b'][i] + ' <s>'
#     for bert format
#     text_reply = '[CLS] ' + df_dev_result['text_a'][i] + ' [SEP] ' + df_dev_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [265]:
predictions, raw_outputs = model.predict(to_predict)

I0602 19:25:43.147072 139786362763072 classification_model.py:801]  Converting to features started. Cache is not used.


In [266]:
total_predict = []
for eid, row in enumerate(raw_outputs):
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
    total_predict.append(predict_class)

In [267]:
df_dev['categories'] = total_predict

In [268]:
df_dev.head()

,idx,text,reply,categories
0,32000,"Drop your cash app, use hashtag #BailoutHumansNow",$tyratomaro #BailoutHumans,"[applause, dance, good_luck, please, thank_you..."
1,32001,After interviewing with a few incredible peopl...,CONGRATS!!!!!,"[agree, applause, high_five, slow_clap, win, yes]"
2,32002,I know GTC festival not happening next month b...,,"[agree, idk, no, omg, scared, shocked]"
3,32003,"Lordy, my daughter just said, “I wonder how th...",,"[eww, facepalm, oh_snap, omg, scared, shocked]"
4,32004,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...,"[facepalm, oops, seriously, shrug, sigh, smh]"


In [269]:
df_dev.to_json('./results/dev.json', orient='records', lines=True)